# Desafio Latam

### 1. ¿Cómo se distribuyen los datos? ¿Qué te llama la atención o cuál es tu conclusión sobre esto?



Para comenzar el análisis de los datos, abrimos la base de datos "Dataset_scl" mediante la función read_csv de Pandas. Posterior a esto se pasa una función para describir la base de datos y conocer la distribución de sus variables. Se especifíca que se quiere utilizar todo tipo de datos para que no nos entregue como resultado solamente las columnas de tipo "int".

In [2]:
# Imported C:\Users\Matias U\Documents\flight-delays-cl\dataset_SCL.csv
import pandas as pd
import numpy as np
dataset_SCL = pd.read_csv(r'C:\Users\Matias U\Documents\flight-delays-cl\dataset_SCL.csv')

C:\Users\Matias U\AppData\Local\Temp\ipykernel_10588\645654622.py:4: DtypeWarning: Columns (1,6) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset_SCL = pd.read_csv(r'C:\Users\Matias U\Documents\flight-delays-cl\dataset_SCL.csv')


In [3]:
dataset_SCL.describe(include='all')

,Fecha_I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha_O,Vlo-O,Ori-O,Des-O,Emp-O,DIA,MES,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES
count,68206,68206,68206,68206,68206,68206,68205,68206,68206,68206,68206.00000,68206.000000,68206.000000,68206,68206,68206,68206,68206
unique,53252,750,1,64,30,62774,774,1,63,32,NaN,NaN,NaN,7,2,23,1,62
top,28-07-17 13:30,174,SCEL,SCFA,LAN,05-11-17 14:51,174,SCEL,SCFA,LAN,NaN,NaN,NaN,Viernes,N,Grupo LATAM,Santiago,Buenos Aires
freq,6,686,68206,5787,37611,5,686,68206,5786,20988,NaN,NaN,NaN,10292,36966,40892,68206,6335
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.71479,6.622585,2017.000029,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00000,1.000000,2017.000000,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.00000,3.000000,2017.000000,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.00000,7.000000,2017.000000,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.00000,10.000000,2017.000000,NaN,NaN,NaN,NaN,NaN


Al ser en su mayoría datos de tipo "string" y fechas, no se puede obtener una tabla descriptiva con todos los datos que otorga la función describe, pero sí nos permite describir la frecuencia y el tipo de observación que más se repite dentro de las variables, otorgandonos información importante respecto a, por ejemplo, el tipo de vuelo más común dentro de los casos observados y le empresa a cargo de los vuelos.

A modo general, se puede comentar que...

### 2. Genera las columnas adicionales y luego expórtelas en un archivo synthetic_features.csv :
### - temporada_alta : 1 si Fecha-I está entre 15-Dic y 3-Mar, o 15-Jul y 31-Jul, o 11-Sep y 30-Sep, 0 si no. 
### - dif_min : diferencia en minutos entre Fecha-O y Fecha-I . 
### - atraso_15 : 1 si dif_min > 15, 0 si no. 
### - periodo_dia : mañana (entre 5:00 y 11:59), tarde (entre 12:00 y 18:59) y noche (entre 19:00 y 4:59), en base a Fecha-I . 

Partimos la segunda parte del desafío llamando el paquete "Mito" que nos facilita el trabajo con bases de datos y el proceso de data wrangling mediante el uso de spreadsheets dentro de Python. Partimos generando la columna dif_min, que indica la diferencia en minutos entre las variables Fecha-O y Fecha-I. El código asociado a las operaciones efectuadas con Mito sobre la base de datos para esta primera tarea aparece a continuación:

In [40]:
# Added column new-column-drl2 to dataset_SCL
dataset_SCL.insert(18, 'new-column-drl2', 0)

# Changed Fecha_I from object to datetime
dataset_SCL['Fecha_I'] = pd.to_datetime(dataset_SCL['Fecha_I'], infer_datetime_format=True, errors='coerce')

# Changed Fecha_I from datetime64[ns] to datetime
dataset_SCL['Fecha_I'] = dataset_SCL['Fecha_I']

# Changed Fecha_O from object to datetime
dataset_SCL['Fecha_O'] = pd.to_datetime(dataset_SCL['Fecha_O'], infer_datetime_format=True, errors='coerce')

# Changed Fecha_O from datetime64[ns] to datetime
dataset_SCL['Fecha_O'] = dataset_SCL['Fecha_O']

# Renamed new-column-drl2 to Diferencia in dataset_SCL
dataset_SCL.rename(columns={'new-column-drl2': 'dif_min'}, inplace=True)

# Set new-column-drl2 in dataset_SCL to =Fecha_O-Fecha_I
dataset_SCL['dif_minute'] = dataset_SCL['Fecha_O']-dataset_SCL['Fecha_I']

Una vez creada la nueva columna pasamos a generar las nuevas variables "temporada_alta", "atraso_15" y "periodo_dia".

In [45]:
import datetime

#dif_min en minutos

dataset_SCL['dif_min'] = dataset_SCL['dif_min'].dt.total_seconds().div(60).astype(int)

# atraso_15

dataset_SCL["atraso_15"] = np.where(dataset_SCL['dif_min'] > 15, "1","0")

# temporada_alta

months = pd.to_datetime(dataset_SCL['Fecha_I'],unit='ms')
 
doy = months.dt.dayofyear

dataset_SCL["temporada_alta"] = pd.cut(doy, bins=(0,62,196,212,254,273,349,366),labels=("1","0","1","0","1","0","1"),ordered=False) 

# periodo_dia 

hours = pd.to_datetime(dataset_SCL['Fecha_I'], format='%H:%M:%S').dt.hour

dataset_SCL['periodo_dia'] = pd.cut(hours,bins=[0,5,12,19,24],include_lowest=True, labels=['19:00 a 4:59','5:00 a 11:59','12:00 a 18:59','19:00 a 4:59'],ordered=False)

# dataset con todas las variables

print(dataset_SCL)


                                        


### 3. ¿Cómo se compone la tasa de atraso por destino, aerolínea, mes del año, día de la semana, temporada, tipo de vuelo? ¿Qué variables esperarías que más influyeran en predecir atrasos?

### 4. Entrena uno o varios modelos (usando el/los algoritmo(s) que prefieras) para estimar la probabilidad de atraso de un vuelo. Siéntete libre de generar variables adicionales y/o complementar con variables externas

### 5. Evalúa tu modelo. ¿Qué performance tiene? ¿Qué métricas usaste para evaluar esa performance y por qué? ¿Por qué elegiste ese algoritmo en particular? ¿Qué variables son las que más influyen en la predicción? ¿Cómo podrías mejorar la performance?


### Conclusiones

### Referencias

- 

- 

- 